# Урок 2. Парсинг HTML. BeautifulSoup
## Домашнее задание

### Установка зависимостей

In [24]:
!~/anaconda3/bin/pip install beautifulsoup4 lxml

### Описание классов

In [23]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

Так как dict() является нехешируемой структурой, то выделим свой data класс и явно объявим ожидаемые поля:

In [24]:
class RecipeData:
    """Data class representing recipe
    """
    
    def __init__(self, **kwargs):
        self.name: str = kwargs.get('name')
        self.ingredients: set = kwargs.get('ingredients')
        self.url: str = kwargs.get('url')
        self.views: int = kwargs.get('views')
        self.comments: int = kwargs.get('comments')
        self.likes: int = kwargs.get('likes')
        self.category: int = kwargs.get('category')
            
    def __hash__(self) -> int:
        return hash(frozenset(self.__dict__.items()))
         
    def __eq__(self, other):
        return (
            self.__class__ == other.__class__ and
            self.__dict__ == other.__dict__
        )
        
    def __repr__(self) -> str:
        return f'{self.__class__.__name__}{self.__dict__}'

Реализация скраппера для povarenok.ru. 

Одна из особенностей сайта заключается в том, что пагинатор создаётся js-скриптом. Выход за пределы максимальной страницы возвразает последнюю. Т.е. если всего страниц 45, то переход на 46, 47 и т.д. страницы возвращает результат 45. Поэтому мы храним в `self._last_recipes` результаты последней обработанной страницы и сравниваем её с вновь полученной. Если результаты одинаковы, то считаем, что распарсили всю категорию.

На вход можно передавать блюда какой кухни мы хотим получить: `it, fr, ch, jp, ru, ua`. Также передаётся `crawl_delay`, чтобы не нагружать сайт.

Архитектура скраппера сделана в виде итератора, который отдаёт элементы по-одному и бесшовно парсит страницы по мере необходимости.

In [72]:
class PovarenokScrapper:
    
    def __init__(self, category: str, crawl_delay: int = 1):
        self._host: str = 'https://www.povarenok.ru/recipes'
        self._category: str = {
            'it': 'kitchen/56',
            'fr': 'kitchen/64',
            'ch': 'kitchen/73',
            'jp': 'kitchen/79',
            'ru': 'kitchen/101',
            'ua': 'kitchen/104',
        }[category]
        self._crawl_delay: int = crawl_delay
        self._current_page_num: int = 0
        self._last_recipes: set = set()
        self._data_to_return: set = set()

    def __iter__(self) -> iter:
        return self

    def __next__(self):
        if len(self._data_to_return) == 0:
            page = self._next_page()
            recipes = self._extract_recipes(page)
            if self._last_recipes != recipes:
                self._last_recipes = recipes
                self._data_to_return = self._last_recipes.copy()
                time.sleep(self._crawl_delay)
                return self.__next__()
            else:
                raise StopIteration
        else:
            return self._data_to_return.pop()

    def __repr__(self) -> str:
        return '{clazz}({sep}host={host},{sep}category={category},{sep}crawl_delay={crawl_delay},\n)'.format(
            sep='\n\t',
            clazz=self.__class__.__name__,
            host=self._host,
            category=self._category,
            crawl_delay=self._crawl_delay,
        )

    def _get_page(self, page_num: int) -> bs:
        url = f'{self._host}/{self._category}/~{page_num}/'
        content = requests.get(url).text
        parsed = bs(content, 'html.parser')
        return parsed
    
    def _next_page(self) -> bs:
        if self._current_page_num is None:
            self._current_page_num = 0
        self._current_page_num += 1
        return self._get_page(self._current_page_num)
    
    def _extract_recipes(self, page: bs) -> set:
        recipes = set()
        for article in page.find_all('article'):
            name = article.select_one('article > h2').get_text().strip()
            url = article.select_one('article > h2 > a')['href']
            ingr = ','.join(map(lambda span: span.get_text().strip().lower(), article.select('div.ingr_fast span')))
            views = article.select_one('ul.icons-wrap .i-views').get_text()
            comm = article.select_one('ul.icons-wrap .i-comments').get_text()
            likes = article.select_one('ul.icons-wrap .i-likes').get_text()
            category = ' > '.join(map(lambda s: s.get_text().replace('  ', ' '), article.select('div.article-breadcrumbs > p > span')))
            recipe = RecipeData(name=name, url=url, ingredients=ingr, views=views, comments=comm, likes=likes, category=category)
            recipes.add(recipe)
        return recipes
    
    def reset(self):
        self._current_page_num = 0
        self._last_recipes = set()

### Практика
Создадим скраппер китайских блюд с `crawl_delay=1` по умолчанию.

In [73]:
scrapper = PovarenokScrapper('ch')
scrapper

PovarenokScrapper(
	host=https://www.povarenok.ru/recipes,
	category=kitchen/73,
	crawl_delay=1,
)

Получим все рецепты и приведём их к формату списка со словарями, чтобы импортировать в pandas.

In [74]:
recipes = [vars(recipe) for recipe in scrapper]

In [75]:
df = pd.DataFrame(recipes)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 673 entries, 0 to 672
Data columns (total 7 columns):
name           673 non-null object
ingredients    673 non-null object
url            673 non-null object
views          673 non-null object
comments       673 non-null object
likes          673 non-null object
category       673 non-null object
dtypes: object(7)
memory usage: 36.9+ KB


In [76]:
df.head(2)

,name,ingredients,url,views,comments,likes,category
0,Огурцы с говядиной по-китайски,"говядина,огурец,лук белый,чеснок,перец болгарс...",https://www.povarenok.ru/recipes/show/89982/,240617,148,2582,Закуски > Закуски из мяса
1,Лепешки с зеленым луком,"мука пшеничная,вода,соль,масло кунжутное,лук з...",https://www.povarenok.ru/recipes/show/101337/,69781,224,3051,Выпечка > Изделия из теста > Лепешки


Чтобы получить больше практической пользы, построим обратный индекс из ингридиентов и поищем блюда, которые мы можем сделать из карпа.

In [77]:
ingridients_per_recipe = list(map(lambda dic: dic['ingredients'].split(','), recipes))
all_ingridients = set([item for sublist in ingridients_per_recipe for item in sublist])
', '.join(all_ingridients)[:500] + '...'

'сок, карп, мука миндальная, глутамат натрия, горчица, кориандр, перец душистый, жидкий дым, крахмал кукурузный, кунжут, чечевица, перец болгарский, рис коричневый, лук зеленый, баклажан, крахмал, масло  растительное, кетчуп, паста арахисовая, соус, тилапия, перец  черный, взбитые сливки, паприка  сладкая, чай черный, мисо-паста, кола, сыр голландский, пиво темное, тунец, ассорти овощное, сырок плавленый, дайкон, кумин, бобы, соевые проростки, вишня, батон, сельдерей корневой, разрыхлитель теста,...'

In [79]:
reverted_index = dict([(i, []) for i in all_ingridients])
for recipe in recipes:
    ingridients = set(recipe['ingredients'].split(','))
    for ingridient in ingridients:
        reverted_index[ingridient].append(recipe['name'])

In [80]:
reverted_index['карп']

['Карп "Белка"',
 'Карп в кисло-сладком соусе',
 'Карп "Хризантема" под соусом',
 'Фаршированный карп в азиатском стиле',
 'Запеченый карп в азиатском стиле',
 'Карп "Белка"',
 'Карп в кисло-сладком соусе',
 'Рыба по-сычуаньски в остром соусе']